# Heuristics - Threshold Strategy

## Imports


In [1]:
import pandas as pd
import numpy as np

# Pandas display options (optional)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.float_format', lambda x: f'{x:.4f}')


## Preparing the data


### Add a new column

In [6]:
df = pd.read_csv("Dataset.csv")

# add a new column 'Timestamp' by extracting the second time from 'Time interval (CET/CEST)'
df["Timestamp"] = pd.to_datetime(
    df["Time interval (CET/CEST)"].str.split(" - ").str[1], 
    dayfirst=True
)

# price
df["Price"] = df["Price"].astype(float)

# sort by 'Timestamp' and reset index
df = df.sort_values("Timestamp").reset_index(drop=True)

df.head()

,Time interval (CET/CEST),Price,Timestamp
0,01.02.2021 00:00 - 01.02.2021 00:15,1033.3270,2021-02-01 00:15:00
1,01.02.2021 00:15 - 01.02.2021 00:30,554.0033,2021-02-01 00:30:00
2,01.02.2021 00:30 - 01.02.2021 00:45,813.4683,2021-02-01 00:45:00
3,01.02.2021 00:45 - 01.02.2021 01:00,774.1577,2021-02-01 01:00:00
4,01.02.2021 01:00 - 01.02.2021 01:15,617.8596,2021-02-01 01:15:00


### Split de dataset

In [8]:
# ============================
# TRAIN / TEST SPLIT
# ============================

# Data de split: până la jumătatea anului 2024 pentru training
SPLIT_DATE = pd.Timestamp("2024-07-01")

train_df = df[df["Timestamp"] < SPLIT_DATE].copy()
test_df  = df[df["Timestamp"] >= SPLIT_DATE].copy()

print("Train size:", len(train_df))
print("Test size :", len(test_df))


Train size: 119615
Test size : 46273


### Extract the percentiles

In [9]:
# ============================
# PERCENTILES FROM TRAIN
# ============================

P1  = train_df["Price"].quantile(0.01)
P10 = train_df["Price"].quantile(0.10)
P90 = train_df["Price"].quantile(0.90)
P99 = train_df["Price"].quantile(0.99)

print("Percentile values:")
print("P1  :", P1)
print("P10 :", P10)
print("P90 :", P90)
print("P99 :", P99)


Percentile values:
P1  : -5222.451723310444
P10 : -978.9433111606559
P90 : 2766.680806251856
P99 : 5278.275952360935


### Batery Init

In [ ]:
# ============================
# BATTERY PARAMETERS
# ============================

SOC_MIN = 0.0
SOC_MAX = 10.0
SOC_START_DAY = 5.0
MIN_TR = 0.1   # tranzacție minimă impusă de competiție

## Heuristic